In [1]:
#导包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = "SimHei" #解决中文乱码问题
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor

In [2]:
df_test = pd.read_csv(r'../DataMining/data_format1\test_format1.csv')
user_info = pd.read_csv(r'../DataMining/data_format1\user_info_format1.csv')
user_log = pd.read_csv(r'../DataMining/data_format1\user_log_format1.csv')

In [3]:
#使用空值去替换
user_info['age_range'].replace(0.0,np.nan,inplace=True)
user_info['gender'].replace(2.0,np.nan,inplace=True)
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    424170 non-null  int64  
 1   age_range  329039 non-null  float64
 2   gender     407308 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB


In [4]:
user_info['age_range'].replace(np.nan,-1,inplace=True)
user_info['gender'].replace(np.nan,-1,inplace=True)
# user_info['age_range'].replace(np.nan,1,inplace=True)
# user_info['gender'].replace(np.nan,0,inplace=True)

In [5]:
#特征值合并

df_test = pd.merge(df_test,user_info,on="user_id",how="left")
 
total_logs_temp = user_log.groupby([user_log["user_id"],user_log["seller_id"]])["item_id"].count().reset_index()
 
total_logs_temp.rename(columns={"seller_id":"merchant_id","item_id":"total_item_id"},inplace=True)
 
df_test = pd.merge(df_test,total_logs_temp,on=["user_id","merchant_id"],how="left")
 
unique_item_id = user_log.groupby(["user_id","seller_id","item_id"]).count().reset_index()[["user_id","seller_id","item_id"]]
 
unique_item_id_cnt = unique_item_id.groupby(["user_id","seller_id"]).count().reset_index()
 
unique_item_id_cnt.rename(columns={"seller_id":"merchant_id","item_id":"unique_item_id"},inplace=True)
 
df_test = pd.merge(df_test, unique_item_id_cnt, on=["user_id", "merchant_id"], how="left")
 
cat_id_temp = user_log.groupby(["user_id", "seller_id", "cat_id"]).count().reset_index()[["user_id", "seller_id", "cat_id"]]
 
cat_id_temp_cnt = cat_id_temp.groupby(["user_id", "seller_id"]).count().reset_index()
 
cat_id_temp_cnt.rename(columns={"seller_id":"merchant_id","cat_id":"total_cat_id"},inplace=True)
 
df_test = pd.merge(df_test, cat_id_temp_cnt, on=["user_id", "merchant_id"], how="left")
 
time_temp = user_log.groupby(["user_id", "seller_id", "time_stamp"]).count().reset_index()[["user_id", "seller_id", "time_stamp"]]
 
time_temp_cnt = time_temp.groupby(["user_id", "seller_id"]).count().reset_index()
 
time_temp_cnt.rename(columns={"seller_id":"merchant_id","time_stamp":"total_time_temp"},inplace=True)
 
df_test = pd.merge(df_test, time_temp_cnt, on=["user_id", "merchant_id"], how="left")
 
click_temp = user_log.groupby(["user_id", "seller_id", "action_type"])["item_id"].count().reset_index()
 
click_temp.rename(columns={"seller_id":"merchant_id","item_id":"times"},inplace=True)
 
click_temp["clicks"] = click_temp["action_type"] == 0
 
click_temp["clicks"] = click_temp["clicks"] * click_temp["times"]
 
click_temp["shopping_cart"] = click_temp["action_type"] == 1
click_temp["shopping_cart"] = click_temp["shopping_cart"] * click_temp["times"]
 
click_temp["purchases"] = click_temp["action_type"] == 2
click_temp["purchases"] = click_temp["purchases"] * click_temp["times"]
 
click_temp["favourites"] = click_temp["action_type"] == 3
click_temp["favourites"] = click_temp["favourites"] * click_temp["times"]
 
four_features = click_temp.groupby(["user_id", "merchant_id"]).sum().reset_index()
 
#删除相关列
four_features = four_features.drop(["action_type", "times"], axis=1)
 
#合并
df_test = pd.merge(df_test, four_features, on=["user_id", "merchant_id"], how="left")
 
#缺失值向前填充
df_test = df_test.fillna(method="ffill")
 

In [6]:
df_test

,user_id,merchant_id,prob,age_range,gender,total_item_id,unique_item_id,total_cat_id,total_time_temp,clicks,shopping_cart,purchases,favourites
0,163968,4605,NaN,-1.0,0.0,2,1,1,1,1,0,1,0
1,360576,1581,NaN,2.0,-1.0,10,9,4,1,5,0,5,0
2,98688,1964,NaN,6.0,0.0,6,1,1,1,5,0,1,0
3,98688,3645,NaN,6.0,0.0,11,1,1,1,10,0,1,0
4,295296,3361,NaN,2.0,1.0,50,8,4,5,47,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
261472,228479,3111,NaN,6.0,0.0,5,2,1,2,4,0,1,0
261473,97919,2341,NaN,8.0,1.0,2,1,1,1,1,0,1,0
261474,97919,3971,NaN,8.0,1.0,16,5,2,3,12,0,4,0
261475,32639,3536,NaN,-1.0,0.0,3,2,1,1,2,0,1,0


In [7]:
#测试数据预处理
# y = df_train["label"]
X = df_test.drop(["user_id", "merchant_id", "prob"], axis=1)
# X['age_range'].replace(-1,3,inplace=True)
# X['gender'].replace(-1,0,inplace=True)
X

,age_range,gender,total_item_id,unique_item_id,total_cat_id,total_time_temp,clicks,shopping_cart,purchases,favourites
0,-1.0,0.0,2,1,1,1,1,0,1,0
1,2.0,-1.0,10,9,4,1,5,0,5,0
2,6.0,0.0,6,1,1,1,5,0,1,0
3,6.0,0.0,11,1,1,1,10,0,1,0
4,2.0,1.0,50,8,4,5,47,0,1,2
...,...,...,...,...,...,...,...,...,...,...
261472,6.0,0.0,5,2,1,2,4,0,1,0
261473,8.0,1.0,2,1,1,1,1,0,1,0
261474,8.0,1.0,16,5,2,3,12,0,4,0
261475,-1.0,0.0,3,2,1,1,2,0,1,0


In [8]:
#将构建好的特征保存
X.to_csv("test_data.csv",index=None)